In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
%matplotlib qt

try:
    from tkinter import Tk
    from tkFileDialog import askopenfilenames
except:
    from tkinter import Tk
    from tkinter import filedialog

In [2]:
def gaussian(x, N, mu, sigma):
    return N*np.exp(-0.5*((x-mu)/sigma)**2)

def doubleGaussian(x, N1, mu1, sigma1, N2, mu2, sigma2):
    return N1*np.exp(-0.5*((x-mu1)/sigma1)**2) + N2*np.exp(-0.5*((x-mu2)/sigma2)**2) 


#### Calibration

In [3]:
root = Tk()
root.withdraw() 
root.wm_attributes('-topmost', 1) 
calib_file_name = filedialog.askopenfilenames(parent = root) 

In [4]:

chanel_cal, energy_cal = np.loadtxt(calib_file_name[0], unpack = 'True')

#NB: Ordered from n-th order to 0-th order
calib_coeff = np.polyfit(chanel_cal, energy_cal, deg = 1)

In [5]:
mock_channel = np.linspace(0,2000,2000)
cal_line = mock_channel*calib_coeff[0] + calib_coeff[1]

fig = plt.figure()
ax = fig.add_subplot()

ax.set_title('Calibration Plot')
ax.plot(mock_channel, cal_line, color = 'grey', label = 'Cal. Line')
ax.scatter(chanel_cal, energy_cal, marker = '+', color ='red', label = 'Cal. Points')
ax.set_xlabel('Channel')
ax.set_ylabel('Energy [keV]')
ax.legend()
ax.grid()


#### Data Analysis

In [13]:
root = Tk()
root.withdraw() 
root.wm_attributes('-topmost', 1) 
data_file_name = filedialog.askopenfilenames(parent = root) 

In [14]:
chanel, counts = np.loadtxt(data_file_name[0], unpack = True, skiprows=10, delimiter =';', usecols=(0,1))
calib_energies =  chanel*calib_coeff[0] + calib_coeff[1]


In [15]:
fig, ax = plt.subplots(1,2, figsize = (15,7))
#ax = fig.add_subplot(nrows = 1,ncols = 2)
plt.suptitle(data_file_name[0])

ax[0].step(chanel, counts, color = 'gray')
ax[0].set_xlabel('Channel')
ax[0].set_ylabel('Counts')
ax[0].grid()

ax[1].step(calib_energies, counts, color = 'gray')
ax[1].set_xlabel('Energy [keV]')
ax[1].set_ylabel('Counts')
ax[1].grid()

In [28]:
min_energy = 600
max_energy = 700

mask = np.logical_and(calib_energies >= min_energy, calib_energies <= max_energy)
energies_to_fit = calib_energies[mask]
counts_to_fit = counts[mask]

fig, ax = plt.subplots(1,1, figsize = (9,7))
ax.step(energies_to_fit, counts_to_fit, color = 'black', label = 'Data')
ax.set_xlabel('Energy [keV]')
ax.set_ylabel('Counts')
ax.grid()

#def doubleGaussian(x, N1, mu1, sigma1, N2, mu2, sigma2):
p0 = [np.max(counts_to_fit), np.min(energies_to_fit), 1, 
      np.max(counts_to_fit), np.min(energies_to_fit), 1]
bounds = [(0,min(energies_to_fit),0,0,min(energies_to_fit),0),
          (np.inf, max(energies_to_fit),np.inf, np.inf, max(energies_to_fit), np.inf)]

par, cov = curve_fit(doubleGaussian, xdata = energies_to_fit, ydata = counts_to_fit, p0 = p0, bounds = bounds)
errs = np.sqrt(np.diag(cov))

x_fit = np.linspace(np.min(energies_to_fit), np.max(energies_to_fit),1000)
y_fit = doubleGaussian(x_fit, *par)

ax.plot(x_fit, y_fit, color = 'red', label = 'Fit')
ax.legend()

print('Data File: ', data_file_name[0])
print('Calibration: ', calib_file_name[0])
print('\n')
print('Fitting energy range: {0} - {1} keV'.format(min_energy, max_energy))
print('Fitting model: double gaussian')
print('\n')
print('FIT PARAMETERS:')
print('N1:     {0:.2f} ± {1:.2f} counts'.format(par[0],errs[0]))
print('Mu1:    {0:.2f} ± {1:.2f} keV'.format(par[1],errs[1]))
print('Sigma1: {0:.2f} ± {1:.2f} keV'.format(par[2],errs[2]))
print('N2:     {0:.2f} ± {1:.2f} counts'.format(par[3],errs[3]))
print('Mu2:    {0:.2f} ± {1:.2f} keV'.format(par[4],errs[4]))
print('Sigma2: {0:.2f} ± {1:.2f} keV'.format(par[5],errs[5]))

Data File:  C:/Users/lisaf/Desktop/due2lab/calib_cesium_137
Calibration:  C:/Users/lisaf/Desktop/GitHub/d2l_spectrometer/calib.txt


Fitting energy range: 600 - 700 keV
Fitting model: double gaussian


FIT PARAMETERS:
N1:     58.74 ± 2.37 counts
Mu1:    642.29 ± 1.54 keV
Sigma1: 17.05 ± 0.95 keV
N2:     130.40 ± 5.69 counts
Mu2:    661.27 ± 0.16 keV
Sigma2: 6.58 ± 0.27 keV
